In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_data(csv_path):
    df = pd.read_csv(csv_path, header=None)
    data = df.values
    X = data[:, 1:].astype(np.float32) / 255.0  # Normalizing pixel values
    y = to_categorical(data[:, 0], num_classes=7)  # Converting labels to one-hot encoding
    X = X.reshape(-1, 48, 48, 1)  # Reshape for CNN input
    return X, y

X_train, y_train = load_data('train_data.csv')
X_val, y_val = load_data('validation_data.csv')

In [12]:
# Simplified Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1), padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.4),

    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(7, activation='softmax')
])

# Optimizer and Callbacks
optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=[reduce_lr, early_stopping]
)

# Save the trained model
model.save('simplified_emotion_model_from_csv.h5')

/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 36s 79ms/step - accuracy: 0.2335 - loss: 2.1760 - val_accuracy: 0.2903 - val_loss: 1.7744 - learning_rate: 0.0100
Epoch 2/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 35s 76ms/step - accuracy: 0.3222 - loss: 1.7522 - val_accuracy: 0.3909 - val_loss: 1.5747 - learning_rate: 0.0100
Epoch 3/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 36s 79ms/step - accuracy: 0.3566 - loss: 1.6681 - val_accuracy: 0.4159 - val_loss: 1.5000 - learning_rate: 0.0100
Epoch 4/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 36s 80ms/step - accuracy: 0.3726 - loss: 1.6182 - val_accuracy: 0.4249 - val_loss: 1.4795 - learning_rate: 0.0100
Epoch 5/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 36s 81ms/step - accuracy: 0.4038 - loss: 1.5540 - val_accuracy: 0.4227 - val_loss: 1.5007 - learning_rate: 0.0100
Epoch 6/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 35s 78ms/step - accuracy: 0.4178 - loss: 1.5170 - val_accuracy: 0.4469 - val_loss: 1.4271 - learning_rate: 0.0100
Epoch 7/100
452/452 ━━━━━━━━━━━━━━━━━━━━ 36s 79ms/step - accuracy: 0.4

In [ ]:
# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Enable optimizations to reduce size
tflite_model = converter.convert()

# Save the converted model
with open('simplified_emotion_model_from_csv.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/ys/2y48tj0d2p7fy2jpz1fhy7qh0000gn/T/tmpa9l6fgyg/assets


INFO:tensorflow:Assets written to: /var/folders/ys/2y48tj0d2p7fy2jpz1fhy7qh0000gn/T/tmpa9l6fgyg/assets


In [4]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the Keras model
model_path = 'simplified_emotion_model_from_csv.h5'
model = load_model(model_path)

# Save the model in the TensorFlow SavedModel format
saved_model_path = 'simplified_emotion_model_from_csv.keras'
model.export(saved_model_path)


INFO:tensorflow:Assets written to: simplified_emotion_model_from_csv.keras/assets


INFO:tensorflow:Assets written to: simplified_emotion_model_from_csv.keras/assets


Saved artifact at 'simplified_emotion_model_from_csv.keras'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name='input_layer_5')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  13048039728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048040608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048040256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048040080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048039376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048040432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048040784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048041840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048041488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048041312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13048039904: TensorSpec(shape=()

In [5]:
import tensorflow as tf

# Convert the SavedModel to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Enable optimizations to reduce size
tflite_model = converter.convert()

# Save the converted model
with open('simplified_emotion_model_from_csv.tflite', 'wb') as f:
    f.write(tflite_model)


W0000 00:00:1716801069.255636 6355144 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1716801069.256138 6355144 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-05-27 11:11:09.257428: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: simplified_emotion_model_from_csv.keras
2024-05-27 11:11:09.258117: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-05-27 11:11:09.258122: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: simplified_emotion_model_from_csv.keras
2024-05-27 11:11:09.267544: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-05-27 11:11:09.268480: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-05-27 11:11:09.331001: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: simplified_emotion_model_from_csv.ke